To complete this assignment, fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`, as well as your name below.

To make sure everything runs as expected, do the following
- **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart)
- **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

A good introduction to Jupyter notebooks is [here](https://realpython.com/jupyter-notebook-introduction/).


In [ ]:
NAME = ""

---

# HW1 (50 points)

In this assignment we will build classifiers to classify a movie review as positive or negative. Using labeled data from IMDb, we will explore how to tokenize each document, create feature vectors, and implement a few different classifiers. Our goal is to understand the overall process of classification using machine learning, as well as to understand how to measure the impact of different algorithmic choices.

There are spaces below for you to both write code and short answers. In some places, there are tests to check your work, though passing tests does not guarantee full credit. I recommend moving sequentially from top to bottom, getting each step working before moving on to the next.

This assignment will use a number of Python libraries, including `pandas`, `sklearn`, `matplotlib`, `seaborn`, `numpy`, and `scipy`. If you haven't already installed these, you can do so by running this command in this directory: `pip install -r requirements.txt`. Minor variants in the version numbers shouldn't affect things much.

In [ ]:
# imports
from collections import Counter
import copy
import numpy as np
from numpy import array as npa
import matplotlib.pyplot as plt
import math
import pandas as pd
import re
from scipy.sparse import csr_matrix
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB,MultinomialNB

%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})
pd.set_option('display.max_colwidth', 100)

## Read and explore data

First, we'll read in the data, compute some basic statistics over it, and review some syntax of Pandas.

The training data is a tab-separated text file called `train.tsv`. We'll first read it into a Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). If you haven't used Pandas before, it is a handy library to read and manipulate tabular data. [Here](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) is a nice overview.

In [ ]:
# read training data into a Pandas DataFrame.
train_df = pd.read_csv('train.tsv', sep='\t')
train_df

In [ ]:
# so, there are 200 positive and 200 negative documents
train_df.label.value_counts()

In [ ]:
# we can get a column, which is a pandas.Series object, like this:
train_df.text  # or: train_df['text']

In [ ]:
# here's the 11th row
train_df.iloc[10]

In [ ]:
# here's how we can iterate over the DataFrame
for rowi, row in train_df.iterrows():
    print('row number:', rowi)
    print('label:', row['label'])
    print('text:', row['text'][:100])
    break

In [ ]:
# we can add a new column by just assigning one.
# E.g., here we add a column indicating the character length of each document.
train_df['length'] = [len(d) for d in train_df.text]

In [ ]:
# pandas has a pretty rich query language we can use to select rows. e.g.:
train_df[train_df.length > 100]

In [ ]:
train_df[(train_df.length>100) & (train_df.label=='neg')]

Pandas supports [matplotlib](https://matplotlib.org/3.3.3/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py) for plotting. E.g., here we plot a histogram of document lengths.

In [ ]:
plt.figure()
train_df.length.plot.hist(bins=50)
plt.xlabel('length')
plt.show()

Does document length vary by label? Let's plot and see:

In [ ]:
plt.figure()
train_df[train_df.label=='pos'].length.plot.hist(bins=50, label='pos', alpha=.5)
train_df[train_df.label=='neg'].length.plot.hist(bins=50, label='neg', alpha=.5)
plt.legend()
plt.xlabel('length')
plt.show()

Hmm...maybe. It does seem like very short documents are more likely to be positive than negative.

## Featurization

As we discussed in class, a first step in text classification is converting a document into a **feature vector**.

There are many things we can consider:

- Do we store word counts or just binary values (1 if word is present, 0 otherwise)?
- Do we keep punctuation?
- Do we keep capitalization?
- Do we just use words or also phrases?

There's no "best" answer to these questions. It is a tradeoff in the number of unique tokens in our model as well as the frequency with which we see each token in the training data. This will affect things like over/under fitting.

Below, complete the `tokenize` function, which takes as input a string representing a document, and returns a list of strings representing each token in the document. Part of your solution will use the `.split()` function of string objects. Then, we have two boolean flags:

- `ignore_case`: if True, convert the entire string to lowercase.
- `strip_punct`: if True, remove any leading or trailing punctuation for each token. E.g., "!it's?" would become "it's". You can use Python's [regular expression library](https://docs.python.org/3/library/re.html) library to do so. Hint: consider using the `sub` method combined with the `\w` and `\W` word classes.

In [ ]:
def tokenize(document, strip_punct=True, ignore_case=True):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert tokenize(" Hi there! Isn't this fun?", strip_punct=True, ignore_case=True) == ['hi', 'there', "isn't", 'this', 'fun']
assert tokenize(" Hi there! Isn't this fun?", strip_punct=False, ignore_case=True) == ['hi', 'there!', "isn't", 'this', 'fun?']

Next, we can choose a specific tokenization setting and apply it to all documents.

We'll store the results in a new column called `tokens`.

In [ ]:
train_df['tokens'] = [tokenize(d, strip_punct=True, ignore_case=True) for d in train_df.text]
train_df

Next we need to create a feature vector for each document. For now, we'll assume **binary features**, which means for each document we'll store a `dict` where words are keys and values are 1 for each word that exists in the document. 

In [ ]:
def featurize(tokens):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
feats = featurize(tokenize(" Hi there! Isn't this fun? Hi there", strip_punct=True, ignore_case=True))
print(feats)
assert sorted(feats.items()) == [('fun', 1), ('hi', 1), ("isn't", 1), ('there', 1), ('this', 1)]
# note that sets and dicts are unordered; i'm sorting by alpha for testing purposes

Now, we can featurize all the documents and assign to a new column called `raw_features`.

In [ ]:
train_df['raw_features'] = [featurize(t) for t in train_df.tokens]
train_df

A common issue in text classification is that many words occur infrequently. This poses a challenge to any machine learning method -- if we've only seen a word once, we cannot be very confident about whether it correlates with the positive or negative class! First, let's count word frequencies by completing the method below. It takes in a list of `dict` objects, from the `raw_features` column, and returns a [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) object representing the number of documents each word appears in.

In [ ]:
def count_word_document_frequency(dict_list):
    # YOUR CODE HERE
    raise NotImplementedError()
word_counts = count_word_document_frequency(train_df.raw_features)

In [ ]:
# we should have 12,166 unique words
len(word_counts)

In [ ]:
# we can print the most common entries in a Counter like so:
word_counts.most_common(10)

In [ ]:
assert count_word_document_frequency(train_df.raw_features).most_common(5) == [('the', 399), ('and', 392), ('a', 391), ('to', 380), ('of', 376)]

assert len(count_word_document_frequency(train_df.raw_features)) == 12166

Just how common are rare words? Let's plot a histogram to find out.

In [ ]:
plt.figure()
plt.hist(word_counts.values(), bins=100)
plt.xlabel('number of documents a word appears in')
plt.ylabel('number of words')
plt.show()
Counter(word_counts.values()).most_common(5)

**Whoa!** There are 7,571 out of 12,166 words that occur exactly once, and 1,698 that occur exactly twice.

This is very common in text collections. There are very many rare words and a small number of very common terms. This can sometimes be better seen in a `log-log` plot. These data somewhat follow something known as a [Power Law](https://en.wikipedia.org/wiki/Power_law) distribution, which just means this log-log plot is well-approximated by a linear fit.

In [ ]:
# log-log plot of word frequencies
def log_log_plot(word_counts):
    plt.figure()
    plt.loglog(sorted(word_counts.values())[::-1])
    plt.xlabel('word rank')
    plt.ylabel('number of documents a word appears in')
    plt.show()
    
log_log_plot(word_counts)

That long, horizontal segment at the bottom right of the plot represents the 7,571 words that occur exactly once.

The words at the top left are the most common words (`the`, `and`, etc.)

It seems that neither very common nor very rare words should be informative in our model.
- For very rare words, we don't see enough examples to have much confidence in our probability estimates.
- For very common words, if they appear in just about every document, they probably do not correlate with the class label.

We will next create a `vocabulary`, which contains our final set of unique features. To do so, we will remove terms that occur too frequently or too infrequently.

The inputs to `create_vocabulary` are:
- word_counts: the Counter object compute above
- min_count: minimum document count allowed
- max_count: maximum document count allowed

This function should return a `dict` where each key is a word and the value is a unique `int` representing the identifier for that word. We will assign each word its value in alphabetical order, e.g. `{'aardvark': 0, 'beetle': 1, ...}`

In [ ]:
def create_vocabulary(word_counts, min_count=2, max_count=100):
    # YOUR CODE HERE
    raise NotImplementedError()


vocabulary = create_vocabulary(word_counts, min_count=2, max_count=100)
list(vocabulary.items())[:10]

In [ ]:
assert len(vocabulary) == 4519
assert list(vocabulary.items())[:10] == [('0', 0), ('1', 1), ('1/2', 2), ('10', 3), ('10/10', 4), ('100', 5), ('11', 6), ('12', 7), ('15', 8), ('16', 9)]

Now that we've finalized our vocabulary, we need to go back through all our `raw_features` in `train_df` and remove any that are not in this `vocabulary`. We store the result in a new column called `features`.

In [ ]:
def prune_features(vocabulary, raw_feature_dict):
    return {k:v for k,v in raw_feature_dict.items() if k in vocabulary}

train_df['features'] = [prune_features(vocabulary, f) for f in train_df.raw_features]
train_df

In [ ]:
# just as a sanity check, let's plot the word frequency distribution again.
word_counts_pruned = count_word_document_frequency(train_df.features)
log_log_plot(word_counts_pruned)
# yes, we've chopped off the left and right part of the original graph.

## Naive Bayes

OK, now we're finally ready to fit a classifier. Let's start with Bernoulli Naive Bayes. Recall the formula to compute the word probabilities for each class, using smoothing:

$$p(x_{k}=1|y=1) = \frac{\epsilon + \sum_{(x_i, y_i) \in D}1[x_{ik}=1 \wedge y_i=1]}{2 \epsilon + \sum_{(x_i, y_i) \in D} 1[y_i=1]}$$

Commonly, $\epsilon=1$ is used (“plus one” smoothing). 

That is, the probability that word $k$ is present in a positive document is the fraction of positive documents that contain word $k$, modulo the smoothing terms.

To compute this, we'll first create a `dict` where keys are words and values are `p(x_{k}=1|y=1)`. We'll create a separate `dict` for $y=$pos and $y=$neg. You should be able to reuse your `count_word_document_frequency` method to help with this.

**note**: be sure to account for words that only appear in one class. These would only have value $\epsilon$ in the numerator.

In [ ]:
def p_x_given_y(train_df, vocabulary, label, epsilon=0):
    # YOUR CODE HERE
    raise NotImplementedError()

p_x_given_pos = p_x_given_y(train_df, vocabulary, 'pos', epsilon=1)
p_x_given_neg = p_x_given_y(train_df, vocabulary, 'neg', epsilon=1)

In [ ]:
assert round(p_x_given_neg['bad'], 3) == 0.307
assert round(p_x_given_pos['bad'], 3) == 0.139

# sanity check 

Let's look at the top values for each class.

In [ ]:
print('top p(x|pos)')
print(sorted(p_x_given_pos.items(), key=lambda x: -x[1])[:15])

print('top p(x|neg)')
print(sorted(p_x_given_neg.items(), key=lambda x: -x[1])[:15])

It makes sense to see things like `great` and `best` in the positive class and `bad` in the negative class, but both distributions are dominated by very common terms -- e.g., `too` is very common in both classes.

To get a better sense of the relative frequency by class, we can simply subtract $p(x|pos)-p(x|neg)$ to find terms that are relatively more frequent in the positive class.

In [ ]:
word_scores = pd.DataFrame([{'word':w, 'pos_score': p_x_given_pos[w] - p_x_given_neg[w]} for w in vocabulary])
print('positive terms')
display(word_scores.sort_values('pos_score', ascending=False).head(10))

print('negative terms')
display(word_scores.sort_values('pos_score', ascending=True).head(10))

These make more sense, though notice we still have some surprising words in the top 10 (e.g., `also`, `man`).

Let's stop and think some more about what smoothing is doing. Below we try different values of $\epsilon$ and print the top 5 terms for the positive class.

In [ ]:
for e in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]:
    ppp = p_x_given_y(train_df, vocabulary, 'pos', epsilon=e)
    ppp = sorted(ppp.items(), key=lambda x: -x[1])
    print(ppp[:5])

As $\epsilon$ increases to infinity, what value will $p(x|y)$ converge to? To answer this, just return the value in the method below (e.g., `return 0` or `return 100`).

In [ ]:
def convergence_value():
    # return the float value that you think p(x|y) converges to as epsilon approaches infinity.
    # YOUR CODE HERE
    raise NotImplementedError()

Think about this result and how it relates to the event model of Bernoulli Naive Bayes. E.g., a coin flip for each word.

The other quantity we need is $p(y=1)$, the prior probability. Since this is a binary classification task, we know that $p(y=0) = 1-p(y=1)$.

In [ ]:
def compute_prior(df):
    """
    Compute the prior probability p(y=1) given a training set DataFrame.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

prior = compute_prior(train_df)

In [ ]:
assert round(compute_prior(train_df), 1) == 0.5


Now, let's actually classify a document using Naive Bayes. Recall the classification formula:

$$
p(y=1|\vec{x}) = \frac{p(y=1)\prod_j p(x_{ij}|y=1)}{p(\vec{x})}
$$

The product in the numerator must loop over all words in the vocabulary. Recall that:

$$
p(x_{ij}=0 \mid y=1) = 1 - p(x_{ij}=1 \mid y=1)
$$

Also, to prevent underflow, we'll have to use this equivalence:
$$
\log(p(y=1|\vec{x})) = \log(p(y=1)) + \sum_j \log p(x_{ij}|y=1) - \log(p(\vec{x}))
$$

where

$$\log(p(x)) = \log \Big( p(\vec{x}|y=1)p(y=1) + p(\vec{x}|y=0)p(y=0) \Big)$$

and to get the final answer:
$$p(y=1|\vec{x}) = \exp \Big( \log(p(y=1|\vec{x})) \Big)$$
(using `math.log` and `math.exp`)


Below, implement `log_p_y_given_x_numerator` which computes just the numerator:

$$\log(p(y)) + \sum_j \log p(x_{ij}|y)$$

Depending on what is passed in for `p_x_given_y` and `prior`, this will compute the numerator either for $y=1$ or $y=0$.

This function is then used by `pr_pos_given_x`, which is done for you, to compute $p(y=1|x)$



In [ ]:
def log_p_y_given_x_numerator(features, p_x_given_y, prior, vocabulary):
    """
    returns log(p(y)) + \sum_j \log p(x_j|y). This is the numerator in the equation
    for p(y|x) above.
    
    note that p_x_given_y and prior can be values for y=1 or y=0, depending on what
    is passed in. see usage in pr_pos_given_x below.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

def pr_pos_given_x(features, p_x_given_pos, p_x_given_neg, prior, vocabulary):
    """
    Returns the probability p(y=1|x). This is complete and should not need to be modified.
    """
    pos_numerator = log_p_y_given_x_numerator(features, p_x_given_pos, prior, vocabulary)
    neg_numerator = log_p_y_given_x_numerator(features, p_x_given_neg, 1-prior, vocabulary)
    # there's an additional log-sum-exp trick to avoid underflow when computing p(x)
    # https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    maxv = max((pos_numerator, neg_numerator))
    log_p_x = maxv + math.log(math.exp(pos_numerator-maxv) + math.exp(neg_numerator-maxv))
    v = pos_numerator - log_p_x
    return math.exp(v)

In [ ]:
assert round(pr_pos_given_x({'great': 1, 'most': 1, 'best': 1, 'well': 1}, p_x_given_pos, p_x_given_neg, prior, vocabulary), 2) == 0.82

In [ ]:
assert round(pr_pos_given_x({'bad': 1, 'worst': 1, 'terrible': 1}, p_x_given_pos, p_x_given_neg, prior, vocabulary), 3) == 0.002

Now, we need to read in the testing data, compute features, and classify.

Here, we have to be very careful to process the data in the exact same way, to ensure our feature set is the same in both training and testing. This means the tokenizer is the same, and the features should be pruned to those in the vocabulary.

In [ ]:
test_df = pd.read_csv('test.tsv', sep='\t')
test_df['tokens'] = [tokenize(d, strip_punct=True, ignore_case=True) for d in test_df.text]
test_df['raw_features'] = [featurize(t) for t in test_df.tokens]
test_df['features'] = [prune_features(vocabulary, f) for f in test_df.raw_features]
test_df.head(2)

Now, we can compute $p(y=1|x)$ for each training and testing instance. We'll store the result in a new column called `pr_pos`:

In [ ]:
train_df['pr_pos'] = [pr_pos_given_x(f, p_x_given_pos, p_x_given_neg, prior, vocabulary)
                      for f in train_df.features]
test_df['pr_pos'] = [pr_pos_given_x(f, p_x_given_pos, p_x_given_neg, prior, vocabulary)
                     for f in test_df.features]
test_df.head(2)

Next, we'll assign the predicted label as positive if the probability is $\ge .5$.

In [ ]:
train_df['predicted_label'] = ['pos' if v >= .5 else 'neg' for v in train_df.pr_pos]
test_df['predicted_label'] = ['pos' if v >= .5 else 'neg' for v in test_df.pr_pos]
test_df.head(2)

Finally, we can compute some quality metrics over the test set. We'll use [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) method from the sklearn library. Please read through the documentation to understand what these metrics are.

In [ ]:
print('results on training data')
print(classification_report(train_df.label, train_df.predicted_label))
print('results on testing data')
print(classification_report(test_df.label, test_df.predicted_label))

So, the overall accuracy is around 78%. Not bad for a simple method!

The precision and recall differ a bit by class, with `pos` having higher precision but lower recall. Based on these results, what is the more common type of error: classifying a negative document as positive (false positive) or classifying a positive document as negative (false negative)? **Submit your answer** by returning either "false positive" or "false negative" in the method below.


In [ ]:
def error_type():
    # return either "false positive" or "false negative"
    # YOUR CODE HERE
    raise NotImplementedError()

## Logistic Regression

Next we'll look more closely at Logistic Regression. Below is the code from class to perform gradient descent, using negative log likelihood as the error function.

In [ ]:
def gradient_descent(gradient_fn, error_fn, theta,
                     learning_rate, D, tolerance, max_iters):
    errori = error_fn(theta, D)
    iters = 0
    trace = [] # for debugging
    while True:
        iters += 1
        theta_cp = copy.copy(theta)
        print('\n\niteration %d' % iters)
        grad = gradient_fn(theta, D)
        trace.append((theta.copy(), grad, errori))
        print('gradient=', grad)
        theta -= learning_rate * grad  # UPDATE!
        newerror = error_fn(theta, D)
        print('old error=%g   new error=%g  theta=%s\n\n' %
              (errori, newerror, str(theta)))
        error_diff = errori - newerror
        # stopping criteria
        if error_diff < 0:
            learning_rate *= .5
            print('error got worse. reducing learning rate to %g' % learning_rate)
            theta = theta_cp
            errori = error_fn(theta, D)
        elif errori - newerror < tolerance:
            print('error change is too small')
            break
        elif iters >= max_iters:
            print('max iterations reached')
            break
        else:
            errori = newerror
    trace = pd.DataFrame(trace, columns=['theta', 'gradient', 'error'])
    display(trace)
    plt.plot(trace.error, 'bo-')
    plt.xlabel('iteration')
    plt.ylabel('error')
    return theta

def f(x, theta):
    # dot product
    return x.dot(theta)

def logistic(x, theta):
    # logistic function: p(y=1|x)
    return 1 / (1 + math.exp(-f(x, theta)))

def nll(theta, D):
    # negative log likelihood
    total = 0
    predictions = [] # for debugging
    for xi, yi in D:
        pred = logistic(xi, theta) if yi==1 else 1-logistic(xi, theta)
        total += math.log(pred)
        predictions.append((xi, yi, pred, 1-pred))
    display(pd.DataFrame(predictions, columns=['x', 'y', 'prediction', 'error']))        
    return -total

def gradient_logistic(theta, D):
    # gradient function for logistic regression
    # updated from lecture to use csr_matrix as feature vectors, 
    # instead of numpy arrays.
    result = np.zeros(len(theta), dtype=np.float64)
    for xi, yi in D:
        p_y_g_x = logistic(xi, theta) if yi==1 else 1-logistic(xi, theta)
        error = yi * (1-p_y_g_x)
        for j, xij in zip(xi.indices, xi.data):
            result[j] += error * xij
    return -result

This code expects the features of a document to be a `numpy array`, rather than a `dict` like we used in naive bayes.

Complete the code below, which creates a numpy array from a feature `dict`. The array should have `1` at location `i` if word `i` is present in the feature dictionary. The order of the array is deterined by the `vocabulary` object.

In [ ]:
def features2array(features, vocabulary):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
feature_vector = features2array({'great': 1, 'terrible': 1}, vocabulary)
assert len(feature_vector) == len(vocabulary)
assert feature_vector[vocabulary['great']] == 1.0
assert feature_vector[vocabulary['terrible']] == 1.0
assert feature_vector[vocabulary['also']] == 0.0

This vector representation is very space inefficient. Since most documents only use a small subset of the full vocabulary, most values will be zero in the feature vector. E.g., below is the number of 0 and 1 values stored in the first training document:

In [ ]:
Counter(features2array(train_df.features[0], vocabulary))

We will instead use a [`csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html), which is a sparse representation of an array. It only stores the non-zero values, along with some indices keeping track of which column each non-zero value correponds to. Below, I've given you the function to do this. We can see that it saves about 33kb just for the first document.

In [ ]:
def features2sparse_array(features, vocabulary):
    return csr_matrix(features2array(features, vocabulary), shape=(1, len(vocabulary)))

dense_array = features2array(train_df.features[0], vocabulary)
sparse_array = features2sparse_array(train_df.features[0], vocabulary)
print('document contains %d/%d words' % (sparse_array.nnz, len(vocabulary)))
print('dense array requires %d bytes, sparse array requires %d bytes' % 
      (dense_array.nbytes, sparse_array.data.nbytes + sparse_array.indices.nbytes + sparse_array.indptr.nbytes))

Now, we will add these sparse arrays to our training and testing DataFrames.

In [ ]:
train_df['feature_vector'] = [features2sparse_array(f, vocabulary) for f in train_df.features]
test_df['feature_vector'] = [features2sparse_array(f, vocabulary) for f in test_df.features]
train_df.head(1)

Next, let's create the full training dataset $D$, which is a list of tuples of the form (feature_vector, label).

In [ ]:
D = [(fv, 1 if label=='pos' else -1) for label, fv in train_df[['label', 'feature_vector']].values]

Now we will create an initial $\theta$ vector of 0s and call gradient descent.

In [ ]:
theta = np.zeros(len(vocabulary))
theta = gradient_descent(gradient_logistic, nll, theta, .3, D, .01, 20)

In [ ]:
# print accuracy on training and testing data
print('results on training data')
train_df['pr_pos_lr'] = [logistic(features2sparse_array(f, vocabulary), theta) for f in train_df.features]
train_df['predicted_label_lr'] = ['pos' if v >= .5 else 'neg' for v in train_df.pr_pos_lr]
print(classification_report(train_df.label, train_df.predicted_label_lr))

print('results on testing data')
test_df['pr_pos_lr'] = [logistic(features2sparse_array(f, vocabulary), theta) for f in test_df.features]
test_df['predicted_label_lr'] = ['pos' if v >= .5 else 'neg' for v in test_df.pr_pos_lr]
print(classification_report(test_df.label, test_df.predicted_label_lr))

We can see that the classifier has perfect accuracy on the training data, and similar accuracy as naive bayes on the test data.

<br><br>

Now, let's inspect the $\theta$ coefficients. Here are the largest and smallest values:

In [ ]:
# print top coefficients for each class.
reverse_vocab = {i:v for v,i in vocabulary.items()}
for i in np.argsort(theta)[::-1][:15]:
    print(reverse_vocab[i], theta[i])
print()
for i in np.argsort(theta)[:15]:
    print(reverse_vocab[i], theta[i])

Inspecting coefficients is a good way to understand your data better. For example, the word "1" appears to be strongly associated with the negative class. Do some digging in the original data to figure out why. In what context does the number "1" appear, and why is it correlated with the negative class? 

**Write your answer in the cell below.**

YOUR ANSWER HERE

But, how should we interpret these coefficients? What does a coefficient of $3.98$ tell us about the term `great`?

In linear regression, we know that the coefficient $\theta_i$ represents the strength of the linear relationship between the independent and dependent variables. That is, as the independent variable ($x_i$) increases by one unit, we expect the dependent variable ($y$) to increase by $\theta_i$.

This is a bit more complicated for logistic regression, since the logistic function introduces a non-linear relationship between $x$ and $y$.

One approach is to just pass the coefficient through the logistic function. This tells us the probability that a document containing just this single word is positive.

In [ ]:
def logistic_single(x):
    return 1 / (1+math.exp(-x))


print("p(y=1|great)=%.3f" % logistic_single(theta[vocabulary['great']]))
print("p(y=1|terrible)=%.3f" % logistic_single(theta[vocabulary['terrible']]))

## sklearn

Now, we did all this the hard way to understand how this works. Of course, there are libraries that do most of these steps for us. E.g., [`sklearn`](https://scikit-learn.org/stable/index.html) is a popular machine learning library; a good tutorial is [here](https://www.datacamp.com/community/tutorials/machine-learning-python).

For example, the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer) class in sklearn provides a way to extract tokens and features from raw text. 

In [ ]:
vec = CountVectorizer(max_df=100, min_df=2, binary=True)
X_train = vec.fit_transform(train_df.text)
X_test = vec.transform(test_df.text)
print('X_train is a csr_matrix with %d rows and %d columns' % (X_train.shape[0], X_train.shape[1]))
print('vec.vocabulary_ maps words to indices. E.g., "great" has index %d' % vec.vocabulary_['great'])

sklearn has a number of classifiers implemented, including [Bernoulli](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html?highlight=bernoulli#sklearn.naive_bayes.BernoulliNB) and [Multinomial](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomial#sklearn.naive_bayes.MultinomialNB) Naive Bayes, and [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression)

In [ ]:
nb = BernoulliNB(alpha=1)
nb.fit(X_train, train_df.label)
y_pred = nb.predict(X_test)
print(classification_report(test_df.label, y_pred))

The accuracy is pretty close to what we got above. There are some small differences in the vocabulary.

We can find the word probabilities in `nb.feature_log_prob`, which stores $\log p(x_i \mid y)$ for each class. We can exponentiate to get the probabilities.

In [ ]:
# these should be similar to what we calculated above in our own implementation.
great_idx = vec.vocabulary_['great']
print('p(great|y=1)=%.2f' % math.exp(nb.feature_log_prob_[1][great_idx]))
print('p(great|y=0)=%.2f' % math.exp(nb.feature_log_prob_[0][great_idx]))

We can similarly fit LogisticRegression.

In [ ]:
# Since we didn't use any regularization in our implementation above,
# I'm setting C to a large number to reduce the effect of the L2 regularizer.
lr = LogisticRegression(C=1e10)
lr.fit(X_train, train_df.label)
y_pred_lr = lr.predict(X_test)
print(classification_report(test_df.label, y_pred_lr))

Our accuracy is pretty close to our implementation above. 

We can find the $\theta$ coefficients in `lr.coef_`.

In [ ]:
print('theta for "great" is %.3f' % lr.coef_[0][great_idx])

## Engineering

Now that you understand a bit more about logistic regression and naive bayes, explore `sklearn` for a while to see if you can come up with an approach that has higher test accuracy than we've shown above. A few ground rules:

- You can use MultinomialNB, BernoulliNB, or LogisticRegression.
- You can modify the tokenization and featurization steps in any way you like.
- You can explore any parameters to the constructors of any of the classifiers.

In the code cells below, try out different settings to see how it affects test accuracy.

In the final written cell, briefly summarize what options you explored, what worked best, what the accuracy was, and why you think your choices improved accuracy.


In [ ]:
# explore here

YOUR ANSWER HERE